In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import xgboost as xgb
import joblib

In [12]:
df = pd.read_csv('data/processed_solar_data.csv', parse_dates=['Timestamp'])

In [13]:
# Инжиниринг признаков
features = ['AirTemperature','RelativeHumidity','WindSpeed','hour','kWp','gen_ma_3h']
X = df[features]
y = df['SolarGeneration']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

# Масштабирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [14]:
# 2. Обучение модели прогнозирования
# Поиск оптимальных параметров
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1]
}

grid = GridSearchCV(
    estimator=xgb.XGBRegressor(objective='reg:squarederror'),
    param_grid=param_grid,
    cv=3,
    scoring='neg_mean_squared_error'
)

grid.fit(X_train_scaled, y_train)
best_model = grid.best_estimator_

In [15]:
# Экспорт модели
joblib.dump(best_model, 'solar_predictor.pkl')
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [16]:
# 3. Детекция аномалий
# Обучение Isolation Forest
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,
    random_state=42
)

iso_forest.fit(X_train_scaled)

# Экспорт детектора
joblib.dump(iso_forest, 'anomaly_detector.pkl')

['anomaly_detector.pkl']

In [18]:
# 4. Оценка производительности
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Прогнозирование
y_pred = best_model.predict(X_test_scaled)

# Вычисление метрик
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5  # Ручное вычисление корня
r2 = r2_score(y_test, y_pred)

print(f"""
Метрики модели:
- MAE: {mean_absolute_error(y_test, y_pred):.2f}
- RMSE: {rmse:.2f}
- R2: {r2:.2f}
""")

# Аномалии
anomaly_scores = iso_forest.decision_function(X_test_scaled)
print(f"Обнаружено аномалий: {(anomaly_scores < 0).sum()} ({(anomaly_scores < 0).mean()*100:.1f}%)")


Метрики модели:
- MAE: 0.73
- RMSE: 1.65
- R2: 0.96

Обнаружено аномалий: 3340 (4.9%)
